In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/ Real-world Deepfake Detection

/content/drive/MyDrive/Colab Notebooks/ Real-world Deepfake Detection


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [ ]:
import timm 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
from timm import create_model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Preprocessing

In [ ]:
class BaseDataset(Dataset):
  def __init__(self, data_path, df):
    self.data_path = data_path
    self.df = df

    self.transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    self.imgs = []
    self.labels = []

    self._load_data()

  def _load_data(self):
    for _, row in self.df.iterrows():
      video = row['video']
      face = row['face']
      label = row['label']

      image_path = os.path.join(self.data_path, video, face)
      img = Image.open(image_path).convert('RGB')
      img = self.transforms(img)

      self.imgs.append(img)
      self.labels.append(label)
    

  def __len__(self):
   return len(self.imgs)

  def __getitem__(self, idx):
    img = self.imgs[idx]
    label = self.labels[idx]
    return img, label

# EDA


In [ ]:
df = pd.read_csv('labels.csv')
data_path = os.getcwd()
testset = BaseDataset(data_path=data_path, df=df)

# Pretrained Model from Before(High, Low Quality)



In [ ]:
model_paths = ['xception_weights_HighQuality-Face2Face.pth', 'efficientnet_weights_HighQuality-Face2Face.pth',
              'xception_weights_HighQuality-NeuralTexture.pth', 'efficientnet_weights_HighQuality-NeuralTexture.pth',
              'xception_weights_LowQuality-Face2Face.pth', 'efficientnet_weights_LowQuality-Face2Face.pth',
              'xception_weights_LowQuality-NeuralTexture.pth', 'efficientnet_weights_LowQuality-NeuralTexture.pth']

model_names = ['xception', 'efficientnet_b0', 'xception', 'efficientnet_b0',
               'xception', 'efficientnet_b0', 'xception', 'efficientnet_b0']

avg_probabilities = []

#load pretrained models
for model_path, model_name in zip(model_paths, model_names):
    full_path = os.path.join('path', model_path)
    model = create_model(model_name, pretrained=False)
    state_dict = torch.load(full_path, map_location=device)
    model_type = model_path.split('_weights_')[0]
    quality = model_path.split('_')[-1].split('.')[0]
    model_type = model_type + quality

    if(model_name == 'xception'):
        in_features = model.fc.in_features
        out_features = 2
        model.fc = torch.nn.Linear(in_features, out_features)
    elif model_name == 'efficientnet_b0':
        num_classes = state_dict['classifier.weight'].size(0)
        state_dict['classifier.weight'] = torch.nn.Parameter(torch.randn(2, state_dict['classifier.weight'].size(1)))
        state_dict['classifier.bias'] = torch.nn.Parameter(torch.randn(2))
        state_dict['classifier.weight'].requiresGrad = True
        state_dict['classifier.bias'].requiresGrad = True
        model.classifier = torch.nn.Linear(state_dict['classifier.weight'].size(1), 2)

    model.load_state_dict(state_dict)
    model = model.to(device).eval()
    print(f'Model: {model_type}')

    probabilities = []
    with torch.no_grad():
        correct_label_probs = []
        for i in range(len(testset)):
            images, labels = testset[i]
            images = images.unsqueeze(0).to(device)
            
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            correct_label_prob = probabilities[0][labels]

            video_folder = testset.df.loc[i, 'video']
            image_name = testset.df.loc[i, 'face']

            print(f'Image {i+1} - Video Folder: {video_folder}, Image Name: {image_name}')
            print(f'Prediction Probability: {correct_label_prob.item()}\n')
            correct_label_probs.append(round(correct_label_prob.item(), 4))

    avg_probability = sum(correct_label_probs) / len(correct_label_probs)
    avg_probabilities.append(avg_probability)
    print(f'Average Prediction Probability: {avg_probability}\n')
    df[model_type] = correct_label_probs

for i in range(len(model_paths)):
  print(f'Model: {model_paths[i]}\n')
  print(f'Average Prediction Probability: {avg_probabilities[i]}\n')


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Model: xceptionHighQuality-Face2Face
Image 1 - Video Folder: video_0, Image Name: face_0.png
Prediction Probability: 0.7213349938392639

Image 2 - Video Folder: video_0, Image Name: face_1.png
Prediction Probability: 0.12337096780538559

Image 3 - Video Folder: video_0, Image Name: face_2.png
Prediction Probability: 1.5875527878961293e-06

Image 4 - Video Folder: video_0, Image Name: face_3.png
Prediction Probability: 0.9999973773956299

Image 5 - Video Folder: video_1, Image Name: face_0.png
Prediction Probability: 1.2663740562857129e-06

Image 6 - Video Folder: video_1, Image Name: face_1.png
Prediction Probability: 2.5088832344977163e-08

Image 7 - Video Folder: video_2, Image Name: face_0.png
Prediction Probability: 0.9996938705444336

Image 8 - Video Folder: video_2, Image Name: face_1.png
Prediction Probability: 0.9999990463256836

Image 9 - Video Folder: video_2, Image Name: face_2.png
Prediction Probability: 0.9999455213546753

Image 10 - Video Folder: video_2, Image Name: face

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Model: xceptionHighQuality-NeuralTexture
Image 1 - Video Folder: video_0, Image Name: face_0.png
Prediction Probability: 0.999982476234436

Image 2 - Video Folder: video_0, Image Name: face_1.png
Prediction Probability: 1.3981732990941964e-05

Image 3 - Video Folder: video_0, Image Name: face_2.png
Prediction Probability: 5.122839183968608e-07

Image 4 - Video Folder: video_0, Image Name: face_3.png
Prediction Probability: 0.999997615814209

Image 5 - Video Folder: video_1, Image Name: face_0.png
Prediction Probability: 1.001276723400224e-05

Image 6 - Video Folder: video_1, Image Name: face_1.png
Prediction Probability: 2.1602631022688e-05

Image 7 - Video Folder: video_2, Image Name: face_0.png
Prediction Probability: 0.9999984502792358

Image 8 - Video Folder: video_2, Image Name: face_1.png
Prediction Probability: 0.999994158744812

Image 9 - Video Folder: video_2, Image Name: face_2.png
Prediction Probability: 0.999985933303833

Image 10 - Video Folder: video_2, Image Name: face_3

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Model: xceptionLowQuality-Face2Face
Image 1 - Video Folder: video_0, Image Name: face_0.png
Prediction Probability: 0.03632248193025589

Image 2 - Video Folder: video_0, Image Name: face_1.png
Prediction Probability: 4.429400632943725e-06

Image 3 - Video Folder: video_0, Image Name: face_2.png
Prediction Probability: 0.002779628150165081

Image 4 - Video Folder: video_0, Image Name: face_3.png
Prediction Probability: 0.9902818202972412

Image 5 - Video Folder: video_1, Image Name: face_0.png
Prediction Probability: 0.00013112829765304923

Image 6 - Video Folder: video_1, Image Name: face_1.png
Prediction Probability: 9.138193490798585e-07

Image 7 - Video Folder: video_2, Image Name: face_0.png
Prediction Probability: 0.9998114705085754

Image 8 - Video Folder: video_2, Image Name: face_1.png
Prediction Probability: 0.9999912977218628

Image 9 - Video Folder: video_2, Image Name: face_2.png
Prediction Probability: 0.9436691403388977

Image 10 - Video Folder: video_2, Image Name: face_

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Model: xceptionLowQuality-NeuralTexture
Image 1 - Video Folder: video_0, Image Name: face_0.png
Prediction Probability: 0.000162462136358954

Image 2 - Video Folder: video_0, Image Name: face_1.png
Prediction Probability: 5.137798780197045e-06

Image 3 - Video Folder: video_0, Image Name: face_2.png
Prediction Probability: 0.9969097971916199

Image 4 - Video Folder: video_0, Image Name: face_3.png
Prediction Probability: 0.5628747344017029

Image 5 - Video Folder: video_1, Image Name: face_0.png
Prediction Probability: 2.5694786017993465e-05

Image 6 - Video Folder: video_1, Image Name: face_1.png
Prediction Probability: 1.6119531665026443e-06

Image 7 - Video Folder: video_2, Image Name: face_0.png
Prediction Probability: 0.9998106360435486

Image 8 - Video Folder: video_2, Image Name: face_1.png
Prediction Probability: 0.999761164188385

Image 9 - Video Folder: video_2, Image Name: face_2.png
Prediction Probability: 0.006169561762362719

Image 10 - Video Folder: video_2, Image Name: 

# Ensemble Models



In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = models

    def forward(self, x):
        outputs = []
        for model in self.models:
            output = model(x)
            outputs.append(output)
        return torch.stack(outputs, dim=0)

model_paths = [os.path.join('path', path) for path in model_paths]

#load pretrained models
models = []
for model_name, model_path in zip(model_names, model_paths):
    model = timm.create_model(model_name, pretrained=False)
    state_dict = torch.load(model_path, map_location=device)
    if model_name == 'xception':
        in_features = model.fc.in_features
        out_features = 2
        model.fc = torch.nn.Linear(in_features, out_features)
    elif model_name == 'efficientnet_b0':
        num_classes = state_dict['classifier.weight'].size(0)
        state_dict['classifier.weight'] = torch.nn.Parameter(torch.randn(2, state_dict['classifier.weight'].size(1)))
        state_dict['classifier.bias'] = torch.nn.Parameter(torch.randn(2))
        state_dict['classifier.weight'].requiresGrad = True
        state_dict['classifier.bias'].requiresGrad = True
        model.classifier = torch.nn.Linear(state_dict['classifier.weight'].size(1), 2)
    model.load_state_dict(state_dict)
    model = model.to(device)
    model.eval()
    models.append(model)

ensemble_model = EnsembleModel(models)

correct_label_probs = []
for i in range(len(testset)):
    image, label = testset[i]
    image = image.unsqueeze(0)
    image = image.to(device)
    outputs = ensemble_model(image)
    probabilities = F.softmax(outputs, dim=2)
    correct_label_prob = probabilities.mean(dim=0)[0, label]
    correct_label_probs.append(correct_label_prob.item())

    print(f"Sample {i+1} - Correct Label Probability: {correct_label_prob.item()}")

df['ensemble'] = correct_label_probs

avg_probability = sum(correct_label_probs) / len(correct_label_probs)
print(f'Average Prediction Probability: {avg_probability}')

df.to_csv('my_prediction.csv', index=False)


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Sample 1 - Correct Label Probability: 0.48454904556274414
Sample 2 - Correct Label Probability: 0.09747490286827087
Sample 3 - Correct Label Probability: 0.25951340794563293
Sample 4 - Correct Label Probability: 0.8031625747680664
Sample 5 - Correct Label Probability: 0.1307603120803833
Sample 6 - Correct Label Probability: 0.07615213096141815
Sample 7 - Correct Label Probability: 0.8633407354354858
Sample 8 - Correct Label Probability: 0.8186813592910767
Sample 9 - Correct Label Probability: 0.737610936164856
Sample 10 - Correct Label Probability: 0.8257976770401001
Sample 11 - Correct Label Probability: 0.7437515258789062
Sample 12 - Correct Label Probability: 0.8482450842857361
Sample 13 - Correct Label Probability: 0.8508270978927612
Sample 14 - Correct Label Probability: 0.7895625233650208
Sample 15 - Correct Label Probability: 0.2833484411239624
Sample 16 - Correct Label Probability: 0.36548101902008057
Sample 17 - Correct Label Probability: 0.6261078715324402
Sample 18 - Correct